In [1]:
import geopandas as gpd
import pandas as pd
import geowrangler.vector_zonal_stats as vzs
import geowrangler.raster_zonal_stats as rzs
import geowrangler.area_zonal_stats as azs
from geowrangler import grids
import folium

import os
import glob
from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Processing generated Isochrone contours

When visualized, the outputed isochrone contours overlap with one another over an area. In this notebook, we aim to assign one travel time contour for each health facility type over an area by getting the minimum travel time. Afterwards, we compute for the population reached in a barangay.

### Input
- Isochrone contour GeoJSON files
- Administrative Boundaries
- WorldPop 2020 Raster

### Output
- Barangay-level dataframe with columns as the travel time and the values are the population reached (sum and percentage) of the narest health facility

## Set-up parameters and directories

In [2]:
PROCESSED_DIR = Path("../../../data/03-processed/")
OUTPUT_DIR = Path("../../../data/04-output/")
GIS_DIR = Path("../../../data/05-gis/")

HOSPITALS_FPATH = Path("../../../data/02-raw/philippines_healthfacilities.gpkg")
ISOCHRONES_DIR = PROCESSED_DIR / "isochrones"
ADMIN_BOUNDS = Path("../../../data/01-admin-bounds/target_admin_bounds.shp")
WP_2020_RASTER = Path("../../../data/02-raw/worldpop/population_count/phl_ppp_2020.tif")

In [3]:
ZOOM_LEVEL = 18  # ~150m
LOCAL_CRS = "EPSG:3123"
PROJ_CRS = "EPSG:4326"

## Load health facilities points

In [4]:
doh_gdf = gpd.read_file(HOSPITALS_FPATH, driver="GPKG")
doh_gdf.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry
0,1.0,DOH000000000002277,Calvario Barangay Health Station,Barangay Health Station,Calvario,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.98987 6.65182)
1,2.0,DOH000000000010319,Cabunbata Barangay Health Station,Barangay Health Station,Cabunbata,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.96630 6.67152)


## Load Isochrones Contours

In [5]:
hospital_files = glob.glob(f"{ISOCHRONES_DIR}/iso_hospital*.geojson")
brgy_healthcenter_files = glob.glob(f"{ISOCHRONES_DIR}/iso_brgy_healthcenters*.geojson")
rhu_files = glob.glob(f"{ISOCHRONES_DIR}/iso_rhu*.geojson")

In [6]:
def concat_health_contours(iso_files):
    """
    Concatenate all the contours for each health facility type
    into one dataframe.
    """
    iso_gdf_list = []

    for file in iso_files:
        gdf = gpd.read_file(file, driver="GeoJSON")
        iso_gdf_list.append(gdf)
    concat_iso = gpd.GeoDataFrame(pd.concat(iso_gdf_list, ignore_index=True))
    concat_iso = concat_iso.set_crs("epsg:4326")
    return concat_iso

In [7]:
hospitals_iso = concat_health_contours(hospital_files)
brgy_healthcenters_iso = concat_health_contours(brgy_healthcenter_files)
rhu_iso = concat_health_contours(rhu_files)

In [8]:
# preview table
rhu_iso

,contour,uid,geometry
0,22,26.0,"POLYGON ((122.07510 6.95991, 122.07089 6.95955..."
1,21,26.0,"POLYGON ((122.07510 6.95811, 122.07044 6.95599..."
2,20,26.0,"POLYGON ((122.07510 6.95544, 122.07309 6.95313..."
3,22,31.0,"POLYGON ((122.09516 6.95940, 122.09316 6.96080..."
4,21,31.0,"POLYGON ((122.09316 6.95831, 122.08831 6.95533..."
...,...,...,...
3089,9,20505.0,"POLYGON ((125.61269 7.08282, 125.61232 7.08088..."
3090,8,20505.0,"POLYGON ((125.61269 7.07991, 125.61169 7.07815..."
3091,10,20824.0,"POLYGON ((125.60323 7.13686, 125.60123 7.13686..."
3092,9,20824.0,"POLYGON ((125.60523 7.13590, 125.60223 7.13574..."


## Load AOI

In [9]:
aoi = gpd.read_file(ADMIN_BOUNDS)  # barangays shapefile
aoi.head(3)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060..."


## Generate grids

In [10]:
bing_tile_grid_generator = grids.BingTileGridGenerator(ZOOM_LEVEL)

In [11]:
%%time
bing_tile_gdf = bing_tile_grid_generator.generate_grid(aoi)

CPU times: user 9min 50s, sys: 142 ms, total: 9min 50s
Wall time: 9min 50s


In [12]:
bing_tile_gdf.shape

(223316, 2)

### Add population to the grids

In [13]:
bing_tile_gdf = rzs.create_raster_zonal_stats(
    bing_tile_gdf,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="population_count"),
    extra_args=dict(nodata=-99999),
)

## Convert contours to grids

In [14]:
def get_min_travel_time(health_iso, facility_type):
    iso_grids = vzs.create_zonal_stats(
        bing_tile_gdf,
        health_iso,
        aggregations=[
            {
                "func": "min",
                "column": "contour",
                "output": f"{facility_type}_travel_time",
            }
        ],
    )
    return iso_grids

In [15]:
%%time

hospital_grids = get_min_travel_time(hospitals_iso, "hospital")
brgy_healthcenter_grids = get_min_travel_time(
    brgy_healthcenters_iso, "brgy_healthcenters"
)
rhu_grids = get_min_travel_time(rhu_iso, "rhu")

CPU times: user 10min 35s, sys: 3.71 s, total: 10min 39s
Wall time: 10min 39s


In [16]:
hospital_grids.head(2)

,quadkey,geometry,population_count,hospital_travel_time
0,132303122020330010,"POLYGON ((121.00616 14.36950, 121.00616 14.370...",59.81089,15.0
1,132303122020330012,"POLYGON ((121.00616 14.36817, 121.00616 14.369...",73.74897,15.0


## Get barangay-level population

In [17]:
%%time

# first to AOI to get total population
aoi_population = rzs.create_raster_zonal_stats(
    aoi,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="brgy_population_count"),
    extra_args=dict(nodata=-99999),
)

CPU times: user 5.09 s, sys: 180 ms, total: 5.27 s
Wall time: 5.26 s


In [18]:
aoi_population.head(5)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_population_count
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1048.389526
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",8001.150879
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",31223.714844
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",2202.831543
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",4607.318359


## Get population reached for each travel time


In [19]:
def compute_population_reached(health_iso_grids, facility_type):
    aoi_population_copy = aoi_population.copy()
    aoi_population_copy = aoi_population_copy.to_crs(LOCAL_CRS)
    health_iso_grids_copy = health_iso_grids.copy()
    health_iso_grids_copy = health_iso_grids_copy.to_crs(LOCAL_CRS)

    suffix = "_travel_time"
    selected_col = [
        col for col in health_iso_grids_copy.columns if col.endswith(suffix)
    ]
    selected_col = selected_col[0]

    # geowrangler way
    brgy_population_count = azs.create_area_zonal_stats(
        aoi_population_copy,
        health_iso_grids_copy,
        [
            dict(func=["count"], column="population_count", output=["count_grids"]),
        ],
        include_intersect=False,
    )

    travel_times = health_iso_grids_copy[selected_col].unique().tolist()
    travel_times = [t for t in travel_times if str(t) != "nan"]
    travel_times = sorted(travel_times)

    for time in travel_times:
        time_col = str(time).split(".")[0]
        brgy_population_count = azs.create_area_zonal_stats(
            brgy_population_count,
            health_iso_grids_copy[health_iso_grids_copy[selected_col] == time],
            [
                dict(
                    func=["sum"],
                    column="population_count",
                    output=[f"pop_reached_total_{time_col}"],
                ),
            ],
            include_intersect=False,
        )
        brgy_population_count[f"pop_reached_pct_{time_col}"] = (
            brgy_population_count[f"pop_reached_total_{time_col}"]
            / brgy_population_count["brgy_population_count"]
        ) * 100

    return brgy_population_count

In [20]:
hospitals_pop_reached = compute_population_reached(hospital_grids, "hospital")
hospitals_pop_reached.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_population_count,...,pop_reached_total_26,pop_reached_pct_26,pop_reached_total_27,pop_reached_pct_27,pop_reached_total_28,pop_reached_pct_28,pop_reached_total_29,pop_reached_pct_29,pop_reached_total_30,pop_reached_pct_30
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((427889.868 1775743.305, 427865.536 1...",1048.389526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((428567.991 1774136.934, 428577.640 1...",8001.150879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
brgy_healthcenter_pop_reached = compute_population_reached(
    brgy_healthcenter_grids, "brgy_healthcenters"
)
brgy_healthcenter_pop_reached.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_population_count,...,pop_reached_total_26,pop_reached_pct_26,pop_reached_total_27,pop_reached_pct_27,pop_reached_total_28,pop_reached_pct_28,pop_reached_total_29,pop_reached_pct_29,pop_reached_total_30,pop_reached_pct_30
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((427889.868 1775743.305, 427865.536 1...",1048.389526,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((428567.991 1774136.934, 428577.640 1...",8001.150879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
rhu_pop_reached = compute_population_reached(rhu_grids, "rhu")
rhu_pop_reached.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_population_count,...,pop_reached_total_26,pop_reached_pct_26,pop_reached_total_27,pop_reached_pct_27,pop_reached_total_28,pop_reached_pct_28,pop_reached_total_29,pop_reached_pct_29,pop_reached_total_30,pop_reached_pct_30
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((427889.868 1775743.305, 427865.536 1...",1048.389526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((428567.991 1774136.934, 428577.640 1...",8001.150879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Change table to long format

In [27]:
def convert_to_long_form(pop_reached_df, facility_type):
    # Create separate DataFrames for 'pop_reached_total' and 'pop_reached_pct'
    df_long_total = pop_reached_df.melt(
        id_vars=["ADM4_PCODE"],
        value_vars=pop_reached_df.filter(like="pop_reached_total").columns,
        var_name="travel_time",
        value_name="pop_reached_total",
    )

    df_long_pct = pop_reached_df.melt(
        id_vars=["ADM4_PCODE"],
        value_vars=pop_reached_df.filter(like="pop_reached_pct").columns,
        var_name="travel_time",
        value_name="pop_reached_pct",
    )

    # Extract 'travel_time' values
    df_long_total["travel_time"] = (
        df_long_total["travel_time"].str.extract(r"_(\d+)").astype(int)
    )
    df_long_pct["travel_time"] = (
        df_long_pct["travel_time"].str.extract(r"_(\d+)").astype(int)
    )

    # Merge the two DataFrames on 'pcode' and 'travel_time'
    result_df = df_long_total.merge(df_long_pct, on=["ADM4_PCODE", "travel_time"])
    result_df = result_df.fillna(0)
    result_df = result_df.round(2)
    result_df.insert(1, "date", "2023-10-25")
    result_df.insert(2, "freq", "S")

    result_copy = result_df.copy()
    result_copy.to_csv(
        OUTPUT_DIR / f"{facility_type}_brgy_population_reached.csv", index=False
    )

    return result_df

In [28]:
hospitals_long = convert_to_long_form(hospitals_pop_reached, "hospitals")
hospitals_long

,ADM4_PCODE,date,freq,travel_time,pop_reached_total,pop_reached_pct
0,PH015518016,2023-10-25,S,5,0.00,0.00
1,PH015518031,2023-10-25,S,5,8139.90,101.73
2,PH015518022,2023-10-25,S,5,29539.28,94.61
3,PH015518024,2023-10-25,S,5,2117.34,96.12
4,PH034919017,2023-10-25,S,5,0.00,0.00
...,...,...,...,...,...,...
22849,PH137401018,2023-10-25,S,30,0.00,0.00
22850,PH137401022,2023-10-25,S,30,0.00,0.00
22851,PH137503007,2023-10-25,S,30,0.00,0.00
22852,PH137503014,2023-10-25,S,30,181.19,0.54


In [29]:
brgy_healthcenter_long = convert_to_long_form(
    brgy_healthcenter_pop_reached, "brgy_healthcenter"
)
brgy_healthcenter_long

,ADM4_PCODE,date,freq,travel_time,pop_reached_total,pop_reached_pct
0,PH015518016,2023-10-25,S,5,112.67,10.75
1,PH015518031,2023-10-25,S,5,7959.48,99.48
2,PH015518022,2023-10-25,S,5,10105.87,32.37
3,PH015518024,2023-10-25,S,5,2117.34,96.12
4,PH034919017,2023-10-25,S,5,205.27,4.46
...,...,...,...,...,...,...
22849,PH137401018,2023-10-25,S,30,0.00,0.00
22850,PH137401022,2023-10-25,S,30,0.00,0.00
22851,PH137503007,2023-10-25,S,30,0.00,0.00
22852,PH137503014,2023-10-25,S,30,85.44,0.25


In [30]:
rhu_long = convert_to_long_form(rhu_pop_reached, "rhu")
rhu_long

,ADM4_PCODE,date,freq,travel_time,pop_reached_total,pop_reached_pct
0,PH015518016,2023-10-25,S,5,0.00,0.00
1,PH015518031,2023-10-25,S,5,751.56,9.39
2,PH015518022,2023-10-25,S,5,3544.00,11.35
3,PH015518024,2023-10-25,S,5,1769.80,80.34
4,PH034919017,2023-10-25,S,5,676.88,14.69
...,...,...,...,...,...,...
22849,PH137401018,2023-10-25,S,30,0.00,0.00
22850,PH137401022,2023-10-25,S,30,0.00,0.00
22851,PH137503007,2023-10-25,S,30,0.00,0.00
22852,PH137503014,2023-10-25,S,30,0.00,0.00
